In [1]:
#importing basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('Training Dataset.arff', header=None, comment='@')
# data = pd.read_csv("urldata.csv")
# data = data.drop(['Domain'], axis = 1).copy()

In [3]:
data.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
dtype: int64

In [4]:
# shuffling the rows in the dataset so that when splitting the train and test set are equally distributed
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,-1,-1,1,-1,-1,-1,0,1,1,1,...,1,1,-1,1,-1,-1,1,1,1,-1
1,1,-1,1,1,1,-1,-1,-1,1,1,...,1,1,1,-1,-1,-1,1,0,1,-1
2,1,-1,1,1,1,-1,0,0,-1,1,...,1,1,1,1,-1,-1,1,0,1,-1
3,1,-1,1,1,1,-1,1,-1,1,1,...,1,1,1,-1,-1,-1,1,1,1,-1
4,1,-1,1,1,1,-1,-1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,1,0,-1,-1


In [5]:
# Sepratating & assigning features and target columns to X & y
# y = data['Label']
# X = data.drop('Label',axis=1)
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [6]:
X.shape, y.shape

column_name=list(X.columns)

In [7]:
# Splitting the dataset into train and test sets: 80-20 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2, random_state = 12)
X_train.shape, X_test.shape

((8844, 30), (2211, 30))

In [8]:
import pickle
import warnings
warnings.filterwarnings('ignore')


dt = pickle.load(open('dt.pkl', 'rb'))
logi = pickle.load(open('logi.pkl', 'rb'))
svm = pickle.load(open('svm.pkl', 'rb'))
rf = pickle.load(open('rf.pkl', 'rb'))
xg = pickle.load(open('xg.pkl', 'rb'))

In [9]:
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

clf = [('dt',dt),('xg',xg),('rf',rf),('svm',svm)] #list of (str, estimator)
stack_model = StackingClassifier( estimators = clf,final_estimator = logi)

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=10, random_state=0, shuffle=True)
results = cross_val_score(stack_model, X_train, y_train, cv=kfold)
print("K-Fold Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))


K-Fold Accuracy: 96.925% (0.584%)


In [10]:
from sklearn.metrics import accuracy_score
stack_model.fit(X_train,y_train)


y_pred=stack_model.predict(X_test)
acc_test= accuracy_score(y_test,y_pred)
print(acc_test*100)

97.69335142469471


In [11]:
from sklearn.metrics import classification_report


print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.98      0.97      0.97       989
           1       0.98      0.98      0.98      1222

    accuracy                           0.98      2211
   macro avg       0.98      0.98      0.98      2211
weighted avg       0.98      0.98      0.98      2211

